<a href="https://colab.research.google.com/github/midhunjmes/midhun-26-02-24/blob/main/Welcome_To_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

hi


In [1]:
pip install presidio-analyzer


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.3/112.3 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 51.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.9/104.9 kB 7.4 MB/s eta 0:00:00


In [2]:
pip install presidio_anonymizer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.9/198.9 kB 19.7 MB/s eta 0:00:00


In [3]:
!pip install spacy


In [4]:
!python -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 1.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [5]:
!pip install geotext spacy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 29.3 MB/s eta 0:00:00


In [14]:
from presidio_analyzer import AnalyzerEngine,PatternRecognizer,Pattern,RecognizerResult
from presidio_anonymizer import AnonymizerEngine
from presidio_analyzer.nlp_engine import NlpEngineProvider
import re
from flair.data import Sentence
from flair.nn import Classifier

In [13]:
ner_tagger = Classifier.load("flair/ner-english-large")

2025-03-10 05:08:44,210 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>


In [38]:
nlp_config = {
    "nlp_engine_name": "spacy",
    "models": [{"lang_code": "en", "model_name": "en_core_web_lg"}]
}
street_address_pattern = Pattern(
    name="street_address_pattern",
    regex=r"\d+\s[A-Za-z]+\s(?:Street|St|Avenue|Ave|Road|Rd|Boulevard|Blvd|Lane|Ln|Drive|Dr|Court|Ct|Place|Pl|Square|Sq)\b",
    score=0.85
)
street_recognizer = PatternRecognizer(
    supported_entity="STREET_ADDRESS",
    patterns=[street_address_pattern]
)



provider = NlpEngineProvider(nlp_configuration=nlp_config)
nlp_engine = provider.create_engine()

analyzer=AnalyzerEngine(nlp_engine=nlp_engine)
anonymizer=AnonymizerEngine()
analyzer.registry.add_recognizer(street_recognizer)

def detect_locations_with_flair(text):
    sentence = Sentence(text)
    ner_tagger.predict(sentence)

    location_entities = []
    for entity in sentence.get_spans("ner"):
        if entity.tag in ["LOC","ORG"]:
            location_entities.append(RecognizerResult(
                entity_type="LOCATION" if entity.tag=="LOC" else "ORGANIZATION",
                start=entity.start_position,
                end=entity.end_position,
                score=entity.score
            ))
    return location_entities

def analyze_and_anonymize(text):
    presidio_results = analyzer.analyze(
        text=text,
        entities=["EMAIL_ADDRESS", "PHONE_NUMBER", "PERSON", "STREET_ADDRESS"],
        language="en"
    )
    flair_results = detect_locations_with_flair(text)

    all_results = presidio_results + flair_results

    anonymized_text = anonymizer.anonymize(text, all_results).text
    return anonymized_text


text='''John Smith and his friend Sarah Johnson recently moved to 789 Elm Street, San Francisco, CA.
John’s email is john.smith@example.com, and Sarah’s email is sarah.johnson@email.net.
They both work at TechCorp, located at 456 Silicon Ave, San Jose, CA.

If you need to reach John, you can call him at +1-415-555-1234.
Sarah’s phone number is (408) 555-5678.

Their friend David Brown, who lives in New York, NY, often visits them.
David’s contact email is david.brown@business.org, and his phone number is 917-555-7890.

Last summer, they took a trip to Los Angeles and stayed at the Grand Sunset Hotel on 12th Avenue.
For hotel reservations, they called +1 310-555-4321.

During the trip, they met Emily White, a researcher from Chicago, IL.
Emily's email is emily.white@university.edu, and her office number is (312) 555-2468.

Later, they traveled to Miami, FL, where they stayed at 234 Ocean Drive, and visited their old college professor, Dr. Robert Williams, whose number is 786-555-1357.

At the end of their journey, they returned home and continued their work at TechCorp.
They frequently travel between different cities for business meetings in Austin, TX, and Seattle, WA.
'''
texts="i'm midhun and i live in kulathur  and I work at TechnoCorp."
#results= analyzer.analyze(text=texts,entities=["EMAIL_ADDRESS","PHONE_NUMBER","PERSON","LOCATION","STREET_ADDRESS"],language="en")
# result_text=anonymizer.anonymize(texts,results)
# print(result_text)
# results= analyzer.analyze(text=text,entities=["EMAIL_ADDRESS","PHONE_NUMBER","PERSON","LOCATION","STREET_ADDRESS"],language="en")
# result_text=anonymizer.anonymize(text,results)
# print(result_text)

anonymized_text = analyze_and_anonymize(text)
print(anonymized_text)


[type: EMAIL_ADDRESS, start: 109, end: 131, score: 1.0, type: EMAIL_ADDRESS, start: 154, end: 177, score: 1.0, type: EMAIL_ADDRESS, start: 452, end: 476, score: 1.0, type: EMAIL_ADDRESS, start: 756, end: 782, score: 1.0, type: PERSON, start: 0, end: 10, score: 0.85, type: PERSON, start: 26, end: 39, score: 0.85, type: STREET_ADDRESS, start: 58, end: 72, score: 0.85, type: PERSON, start: 93, end: 97, score: 0.85, type: PERSON, start: 137, end: 142, score: 0.85, type: STREET_ADDRESS, start: 218, end: 233, score: 0.85, type: PERSON, start: 271, end: 275, score: 0.85, type: PERSON, start: 314, end: 319, score: 0.85, type: PERSON, start: 368, end: 379, score: 0.85, type: PERSON, start: 427, end: 432, score: 0.85, type: PERSON, start: 695, end: 706, score: 0.85, type: STREET_ADDRESS, start: 882, end: 897, score: 0.85, type: PERSON, start: 944, end: 959, score: 0.85, type: PHONE_NUMBER, start: 338, end: 352, score: 0.75, type: PHONE_NUMBER, start: 502, end: 514, score: 0.75, type: PHONE_NUMBE

In [8]:
import json

In [9]:


street_address_pattern = Pattern(
    name="street_address_pattern",
    regex=r"\d+?,\s[A-Za-z]+\s(?:Street|St|Avenue|Ave|Road|Rd|Boulevard|Blvd|Lane|Ln|Drive|Dr|Court|Ct|Place|Pl|Square|Sq)\b",
    score=0.85
)
street_recognizer = PatternRecognizer(
    supported_entity="STREET_ADDRESS",
    patterns=[street_address_pattern]
)
analyzer.registry.add_recognizer(street_recognizer)
def anonymize_json(data):
    if isinstance(data, dict):
        return {key: anonymize_json(value) for key, value in data.items()}
    elif isinstance(data, list):
        return [anonymize_json(item) for item in data]
    elif isinstance(data, str):
        results = analyzer.analyze(text=data, entities=["PERSON", "EMAIL_ADDRESS", "PHONE_NUMBER", "LOCATION","STREET_ADDRESS"], language="en")
        return anonymizer.anonymize(text=data, analyzer_results=results).text if results else data
    else:
        return data
json_data = {
    "name": "Midhun James",
    "email": "mid@gmail.com",
    "phone": "+911234567890",
    "address": "123 Church Street, Kulathoor",
    "messages": [
        "Hello, this is midhun!",
        "My email is mid@gmail.com and I live in Kochi."
    ]
}
anonymized_data = anonymize_json(json_data)
print(json.dumps(anonymized_data, indent=2))

{
  "name": "<PERSON>",
  "email": "<EMAIL_ADDRESS>",
  "phone": "<PHONE_NUMBER>",
  "address": "<STREET_ADDRESS>, <PERSON>",
  "messages": [
    "Hello, this is <PERSON>!",
    "My email is <EMAIL_ADDRESS> and I live in <LOCATION>."
  ]
}


In [10]:
pip install flair regex

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 21.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 64.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.5/139.5 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.6/202.6 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 111.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 91.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [39]:
import re
from flair.data import Sentence
from flair.nn import Classifier

# Load Flair's NER model
ner_tagger = Classifier.load("flair/ner-english-large")

# Function to find email addresses
def find_emails(text):
    email_pattern = r"[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}"
    return [(match.start(), match.end(), "EMAIL") for match in re.finditer(email_pattern, text)]

# Function to find phone numbers
def find_phone_numbers(text):
    phone_pattern = r"\+?\d{1,3}[-.\s]?\(?\d{1,4}\)?[-.\s]?\d{1,4}[-.\s]?\d{1,9}"
    return [(match.start(), match.end(), "PHONE") for match in re.finditer(phone_pattern, text)]

# Function to mask sensitive information
def anonymize_text(text):
    sentence = Sentence(text)

    # Run Flair NER on text
    ner_tagger.predict(sentence)

    # Collect detected entities
    entities = [(e.start_position, e.end_position, e.tag) for e in sentence.get_spans("ner")]

    # Add email and phone number detections
    entities += find_emails(text)
    entities += find_phone_numbers(text)

    # Sort entities by start position (important for replacement)
    entities.sort()

    # Replace entities with masked values
    anonymized_text = list(text)
    for start, end, entity_type in reversed(entities):  # Reverse to avoid index shift
        anonymized_text[start:end] = f"[{entity_type}]"

    return "".join(anonymized_text)

# Example text containing sensitive information
text = """John Smith lives in Trivandrum, Kerala and works at Google.
His email is john.smith@example.com, and his phone number is +91-9876543210.
He recently moved to 456 Park Avenue, New York, NY."""

# Mask sensitive details
anonymized_text = anonymize_text(text)
print(anonymized_text)


2025-03-10 06:16:05,044 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>
[PER] lives in [LOC], [LOC] and works at [ORG].
His email is [EMAIL], and his phone number is [PHONE].
He recently moved to 456 [LOC], [LOC], [LOC].


In [33]:
text="iam midhun"
sentence=Sentence(text)
ner_tagger.predict(sentence)
print(sentence)
print(sentence.get_spans("ner"))

Sentence[2]: "iam midhun"
[]
